Drew Lickman\
CSCI 4820-001\
Project #3\
Due: 10/9/24

AI Usage Disclaimer:


# Lexicon-Based Sentiment Analysis using Custom Logistic Regression

## Assignment Requirements:

### Input
---

- Positive words
- Negative words
- IMDb reviews

### Processing
---

- There are two classifiers
	- Custom Logistic Regression
	- sklearn LogisticRegression
- Implement a Python class (CustomLogisticRegression)
	- \__init\__(self, learning_rate, num_iters) method
		- self.learning_rate
		- self.num_iters
		- self.weights = None
		- self.bias = None
	- sigmoid(z)
		- return result
	- fit(X, y)
		- Sets weights to correct shape and initializes them to 0
		- Applies batch gradient descent to the entire dataset in a loop for num_iters
	- predict(X)
		- z = w dot x + b
		- return sigmoid(z) 

### Output
---

- 

## Python Code

Markdown above each cell used to explain each block of code

1. Load and preprocess IMDb reviews

In [ ]:
import math
import numpy as np
import sklearn

sentimentWords = []
posWords = []
negWords = []
with open("positive-words.txt", encoding="utf-8") as positivewords:
	lines = positivewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '': 
			posWords.append(line.rstrip('\n'))
with open("negative-words.txt", encoding="utf-8") as negativewords:
	lines = negativewords.readlines()
	for line in lines:
		if line[0] != ";" and line.strip() != '':
			negWords.append(line.rstrip('\n'))
sentimentWords = posWords + negWords # Combine positive words and negative words into one array
#print(sentimentWords)

reviews = []
with open("imdb_reviews.txt", encoding="utf-8") as imdbreviews:
    lines = imdbreviews.readlines()
    for line in lines:
        reviews.append(line)
print(reviews[0])

2. Create Features(X) table and Labels(y) array

In [ ]:
X = [[]] 	# Features
y = [] 		# Labels
posCount = 0
negCount = 0

for review in range(5):
	X.append([])
	for word in range(len(sentimentWords)):
		if sentimentWords[word] in reviews[review]:
			#print(sentimentWords[word])
			X[review].append(True)
			posCount += 1 								# need to adjust this to count positive / negative word
														# using posWords and negWords so they can be counted
														# otherwise it will always be 0 in labels
		else:
			X[review].append(False)
			negCount += 1
	if posCount >= negCount:
		y.append(1)
	else:
		y.append(0)
	posCount = 0
	negCount = 0
X.pop() #Remove empty [] at end of X

for review in range(len(X)):
	print(X[review])
	print(y[review])


In [ ]:


class CustomLogisticRegression():
	# Should be 25-30 lines of code
	print("hi")
	def __init__(self, learning_rate, num_iters):
		print("init")
		self.learning_rate = learning_rate
		self.num_iters = num_iters
		self.weights = None
		self.bias = None

	def fit(self, X, y):
		# Train the model using gradient descent
		# X is training features, y is labels
		#np.array(X)
		#np.array(y)
		features = X.shape[1]
		self.weights = np.zeros(features)
		self.bias = 0
		# Sets the weights to the correct shape and initializes them to 0
		# Applies batch gradient descent (entire dataset) in a loop for # iterations specified
		# Re-initialize weights and bias 
		print("fit")

	def sigmoid(self, x):
		print("apply sigmoid to x")
		output = 1/(1+math.pow((math.e), -x))
		print("Sigmoid:", output)
		return output

	def predict(self, X):
		# Make predictions on test data
		# Computes W DOT x + b
		output = np.dot(self.weights, X) + self.bias #might need to swap W X
		output = self.sigmoid(output)
		# Convert to binary output
		output = int(output >= 0.5)
		return output



In [ ]:
sklearn.LogisticRegression(solver='sag', C=0.001, max_iter=500)
CustomLogisticRegression(learning_rate=0.1, num_iters=500)